In [1]:
%run heuristics_helper.ipynb
%run graph_helper.ipynb

In [2]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations
import time
from scipy.stats import beta
import pandas as pd
%matplotlib inline

In [3]:
obj_functions = [obj_disagreement, obj_polarization, obj_sum]

In [4]:
DISAGREEMENT = 0 
POLARIZATION = 1
BOTH = 2

In [5]:
n = 1000
d1 = 0.7 # density within communities
d2 = 0.1 # density of edges between communities
k_range = 1000 #budget
(c1, c2, G) = make_block_matrix(n, d1, d2)

s = make_beta_opinions(5, 2, n, c1, c2) 

# Execute nonadaptive heuristics

In [11]:
# function for performing nonadaptive heuristics
def compare_nonadaptive(n, k, c1, c2, G, s, obj_type):
    L = scipy.sparse.csgraph.laplacian(G, normed=False)
    A = np.linalg.inv(np.identity(n) + L)
    m = num_edges(L, n)
           
    s_5050 = np.copy(s)
    s_25 = np.copy(s)
    
    (to_zero, to_one, max_obj_5050) = choose5050(obj_functions[obj_type], A, L, s_5050, n, m, k, c1, c2)
    
    (to_zero_c1, to_one_c1, to_zero_c2, to_one_c2, max_obj_25) = choose25(obj_functions[obj_type], 
                                                                         A, L, s_25, n, m, k, c1, c2)

    return (max_obj_5050, max_obj_25)

# Testing: Disagreement

In [12]:
(s, s_greedy, max_obj_greedy_arr, s_local_innate, max_obj_local_innate_arr, s_random, 
 max_obj_random_arr, s_partial_random, max_obj_partial_random_arr, s_mean, max_obj_mean_arr,
 s_deg, max_obj_deg_arr, s_w_deg, max_obj_w_deg_arr, s_double_heuristic, 
 max_obj_double_heuristic_arr, original_obj) = compare_algorithms(n, k_range, G, DISAGREEMENT, opinions=s)

max_obj_5050_arr = np.zeros(k_range)
max_obj_25_arr = np.zeros(k_range)
for k in range(k_range):
    (obj5050, obj25) = compare_nonadaptive(n, k, c1, c2, G, s, DISAGREEMENT)
    max_obj_5050_arr[k] = obj5050
    max_obj_25_arr[k] = obj25




In [16]:
objs = [max_obj_greedy_arr, max_obj_mean_arr, max_obj_partial_random_arr, max_obj_random_arr, 
        max_obj_local_innate_arr, max_obj_deg_arr, max_obj_w_deg_arr, max_obj_double_heuristic_arr, 
        max_obj_5050_arr, max_obj_25_arr]

df = pd.DataFrame(objs)
df = df.transpose()
df.columns = labels

In [17]:
df.to_pickle("./sbm_pkl/disagreement_1000_1000_6_1_20_2.pkl")

# Testing: Polarization

In [18]:
(s, s_greedy, max_obj_greedy_arr, s_local_innate, max_obj_local_innate_arr, s_random, 
            max_obj_random_arr, s_partial_random, max_obj_partial_random_arr, s_mean, max_obj_mean_arr,
            s_deg, max_obj_deg_arr, s_w_deg, max_obj_w_deg_arr, s_double_heuristic, 
            max_obj_double_heuristic_arr, original_obj) = compare_algorithms(n, k_range, G, POLARIZATION, opinions=s)

max_obj_5050_arr = np.zeros(k_range)
max_obj_25_arr = np.zeros(k_range)
for k in range(k_range):
    (obj5050, obj25) = compare_nonadaptive(n, k, c1, c2, G, s, POLARIZATION)
    max_obj_5050_arr[k] = obj5050
    max_obj_25_arr[k] = obj25

In [22]:
objs = [max_obj_greedy_arr, max_obj_mean_arr, max_obj_partial_random_arr, max_obj_random_arr, 
        max_obj_local_innate_arr, max_obj_deg_arr, max_obj_w_deg_arr, max_obj_double_heuristic_arr, 
        max_obj_5050_arr, max_obj_25_arr]

df = pd.DataFrame(objs)
df = df.transpose()
df.columns = labels

In [23]:
df.to_pickle("./sbm_pkl/polarization_1000_1000_6_1_20_2.pkl")

# Testing: Sum

In [24]:
(s, s_greedy, max_obj_greedy_arr, s_local_innate, max_obj_local_innate_arr, s_random, 
            max_obj_random_arr, s_partial_random, max_obj_partial_random_arr, s_mean, max_obj_mean_arr,
            s_deg, max_obj_deg_arr, s_w_deg, max_obj_w_deg_arr, s_double_heuristic, 
            max_obj_double_heuristic_arr, original_obj) = compare_algorithms(n, k_range, G, BOTH, opinions=s)

max_obj_5050_arr = np.zeros(k_range)
max_obj_25_arr = np.zeros(k_range)
for k in range(k_range):
    (obj5050, obj25) = compare_nonadaptive(n, k, c1, c2, G, s, BOTH)
    max_obj_5050_arr[k] = obj5050
    max_obj_25_arr[k] = obj25

In [27]:
objs = [max_obj_greedy_arr, max_obj_mean_arr, max_obj_partial_random_arr, max_obj_random_arr, 
        max_obj_local_innate_arr, max_obj_deg_arr, max_obj_w_deg_arr, max_obj_double_heuristic_arr, 
        max_obj_5050_arr, max_obj_25_arr]

df = pd.DataFrame(objs)
df = df.transpose()
df.columns = labels

In [28]:
df.to_pickle("./sbm_pkl/sum_1000_1000_6_1_20_2.pkl")

# Analysis

What sort of vertices are picked by adaptive heuristics with graphs generated using stochastic block model? 

In [15]:
def intersection(a, b):
    return [value for value in a if value in b]

In [16]:
# among the vertices picked, which ones belong to community 1 and which ones belong to community 2?
def find_communities(c1, c2, v, first_k):
    changed1 = intersection(v[:first_k], c1)
    changed2 = intersection(v[:first_k], c2)
    return (changed1, changed2)

In [17]:
# check if node in c1 is connected to anything in c2 and vice versa - call such a node a "fringe" node
# returns the list of nodes in the other community that given vertex v is connected to
def is_fringe_node(c, v, G):
    neighbors = np.where(G[int(v),] != 0)[0]
    return intersection(neighbors, c)

In [56]:
# calculate ratio of # of changed in c1 vs # of changed in c2 (see if there is a consistent imbalance)
a = np.arange(10, 100)
ratio = np.zeros(len(a))
diff = np.zeros(len(a))

for i in a:
    # replace "v_greedy_arr" to analyze other heuristics
    v_greedy_arr = list(set(np.where(s_greedy == 0.0)[0]) | set(np.where(s_greedy == 1.0)[0]))
    (changed1, changed2) = find_communities(c1, c2, v_greedy_arr, i) 
    ratio[i - 10] = len(changed1)/len(changed2)
    diff[i - 10] = np.abs(len(changed1) - len(changed2))
    
print(np.max(ratio))
print(np.min(ratio))
print(np.max(diff))

9.0
1.0204081632653061
9.0


In [57]:
(changed1, changed2) = find_communities(c1, c2, v_greedy_arr, 10)
for v in changed1:
    print("In the other community, {} is connected to {}".format(v, is_fringe_node(c2, v, G)))

print("\n")

for v in changed2:
    print("In the other community, {} is connected to {}".format(v, is_fringe_node(c1, v, G)))


In the other community, 1 is connected to [13, 75, 90, 94, 98, 107, 111, 147, 171, 208, 219, 229, 250, 254, 260, 266, 272, 344, 397, 408, 419, 426, 501, 511, 520, 561, 587, 610, 611, 617, 671, 674, 682, 719, 751, 759, 814, 836, 892, 910, 966, 976, 979, 992]
In the other community, 2 is connected to [17, 70, 121, 131, 141, 147, 153, 157, 191, 204, 205, 208, 215, 271, 288, 304, 311, 315, 333, 346, 358, 360, 375, 382, 383, 394, 399, 484, 489, 511, 514, 519, 529, 558, 571, 588, 589, 610, 626, 653, 662, 671, 693, 714, 760, 789, 797, 810, 815, 825, 837, 868, 877, 888, 904, 906, 935, 939, 987]
In the other community, 3 is connected to [15, 18, 33, 56, 77, 88, 90, 95, 98, 102, 175, 194, 204, 206, 217, 229, 250, 316, 320, 335, 367, 371, 408, 419, 462, 499, 517, 570, 586, 589, 592, 600, 612, 619, 662, 671, 675, 681, 682, 736, 757, 775, 832, 871, 883, 904, 935, 977, 978]
In the other community, 4 is connected to [26, 28, 33, 34, 62, 75, 119, 121, 124, 133, 149, 165, 177, 184, 194, 207, 235, 245, 